In [1]:
#Pray for P100
!nvidia-smi

Thu Nov 28 20:26:38 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!pip install transformers

     |████████████████████████████████| 368kB 2.8MB/s 
     |████████████████████████████████| 645kB 64.3MB/s 
     |████████████████████████████████| 1.0MB 63.7MB/s 
     |████████████████████████████████| 860kB 70.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=b5ffe54f81bac880993da05ff47e3c2e41374eb0fce469a4449540db2c38314c
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [3]:
!pip install gdown
!gdown --id 1WtbXCv3vPB5ql6w0FVDmAEMmWadbrCuG --output dnli.zip
!unzip dnli.zip

Downloading...
From: https://drive.google.com/uc?id=1WtbXCv3vPB5ql6w0FVDmAEMmWadbrCuG
To: /content/dnli.zip
390MB [00:07, 50.7MB/s]
Archive:  dnli.zip
   creating: dnli/
   creating: dnli/dialogue_nli/
  inflating: dnli/dialogue_nli/dialogue_nli_train.jsonl  
  inflating: dnli/dialogue_nli/print_dtypes.py  
  inflating: dnli/dialogue_nli/README.txt  
   creating: __MACOSX/
   creating: __MACOSX/dnli/
   creating: __MACOSX/dnli/dialogue_nli/
  inflating: __MACOSX/dnli/dialogue_nli/._README.txt  
  inflating: dnli/dialogue_nli/dialogue_nli_dev.jsonl  
  inflating: __MACOSX/dnli/dialogue_nli/._dialogue_nli_dev.jsonl  
  inflating: dnli/dialogue_nli/dialogue_nli_test.jsonl  
  inflating: dnli/dialogue_nli/dialogue_nli_verified_test.jsonl  
  inflating: dnli/.DS_Store          
  inflating: __MACOSX/dnli/._.DS_Store  
  inflating: dnli/dialogue_nli.tar.gz  
   creating: dnli/dialogue_nli_extra/
  inflating: dnli/dialogue_nli_extra/dialogue_nli_EXTRA_uu_test.jsonl  
  inflating: dnli/dialogu

In [0]:
# Prepare tsv files as required by DataProcessor class

import json
import pandas as pd
import csv

#Read jsonl files for train/val
with open('dnli/dialogue_nli/dialogue_nli_train.jsonl') as f:
    lines = f.readlines()
train_data = json.loads(lines[0])
with open('dnli/dialogue_nli/dialogue_nli_dev.jsonl') as f:
    lines = f.readlines()
val_data = json.loads(lines[0])

#Make dataframes
df_train = pd.DataFrame.from_dict(train_data, orient='columns')
df_train['index'] = df_train.index
df_train = df_train[['index','sentence1', 'sentence2','label']]
df_train['label'] = df_train['label'].replace(['negative', 'positive'], ['contradiction', 'entailment'])

df_val = pd.DataFrame.from_dict(val_data, orient='columns')
df_val['index'] = df_val.index
df_val = df_val[['index','sentence1', 'sentence2','label']]
df_val['label'] = df_val['label'].replace(['negative', 'positive'], ['contradiction', 'entailment'])

!mkdir dnli_data
df_train.to_csv('dnli_data/train.tsv', sep='\t',quoting=csv.QUOTE_NONE, index=False)
df_val.to_csv('dnli_data/dev.tsv', sep='\t',quoting=csv.QUOTE_NONE, index=False)

In [5]:
import torch
import torch.nn.functional as F

from transformers import RobertaTokenizer
from transformers import RobertaForSequenceClassification, RobertaConfig

#Inference model
mnli_config = RobertaConfig.from_pretrained('roberta-large-mnli')
mnli_tokenizer = RobertaTokenizer.from_pretrained('roberta-large-mnli')
mnli_model = RobertaForSequenceClassification.from_pretrained('roberta-large-mnli')

mnli_model.to('cuda') 

100%|██████████| 1425744429/1425744429 [01:47<00:00, 13237830.61B/s]


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm

In [0]:
def get_predictions(probs):
    #getting confused with the order of labels in mnli...
    #TODO: verify this.
    #C E N for dnli
    #C N E for mnli
    inferences = ['Contradiction', 'Entailment', 'Contradiction']

    confidence, pred = torch.max(probs, 1)
    #print(pred)
    #print(list(zip(confidence, pred, [inferences[k] for k in pred])))

    return confidence, pred

def infer(tokenizer, model, batch_of_pairs, labels, device):

    batch_inputs =  [torch.tensor(tokenizer.encode(*pair,add_special_tokens=True)) for pair in batch_of_pairs]
    
    padded_batch = torch.nn.utils.rnn.pad_sequence(batch_inputs, batch_first=True, padding_value = -1).to(device)
    mask = torch.zeros_like(padded_batch)
    mask[padded_batch!=-1] = 1
    padded_batch[padded_batch==-1] = 0

    #Masking isn't happening properly. Wait for PR to be accepted.
    #https://github.com/huggingface/transformers/issues/1761

    # import pdb
    # pdb.set_trace()

    outputs = model(padded_batch, labels=labels, attention_mask = mask)
    loss, logits = outputs[:2]
    softmax = torch.nn.Softmax(dim=1)
    probs = softmax(logits)

    return probs

In [0]:
batch_of_pairs = df_val[['sentence1', 'sentence2']].values.tolist()
labels = df_val['label'].tolist()
label_map = {'contradiction':0, 'neutral':1, 'entailment':2}

labels = [[label_map[i]] for i in labels]
labels = torch.tensor(labels).to('cuda')

In [0]:
tot_pred = torch.zeros(16500,1).squeeze()
for i in range(0,16500,10):
  probs = infer(mnli_tokenizer, mnli_model, batch_of_pairs[i:i+10], labels[i:i+10], 'cuda')
  confidence, pred = get_predictions(probs)
  tot_pred[i:i+10] = pred

In [20]:
sum(labels.squeeze() == tot_pred.to('cuda')) / 16500.0

tensor(0.6621, device='cuda:0')

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [23]:
dnli_config = RobertaConfig.from_pretrained('drive/My Drive/NLP/project/models/roberta-dnli')
dnli_tokenizer = RobertaTokenizer.from_pretrained('drive/My Drive/NLP/project/models/roberta-dnli')
dnli_model = RobertaForSequenceClassification.from_pretrained('drive/My Drive/NLP/project/models/roberta-dnli')

dnli_model.to('cuda') 

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm

In [0]:
batch_of_pairs = df_val[['sentence1', 'sentence2']].values.tolist()
labels = df_val['label'].tolist()
label_map = {'contradiction':0, 'entailment':1, 'neutral':2}

labels = [[label_map[i]] for i in labels]
labels = torch.tensor(labels).to('cuda')

In [0]:
tot_pred = torch.zeros(16500,1).squeeze()
for i in range(0,16500,10):
  probs = infer(dnli_tokenizer, dnli_model, batch_of_pairs[i:i+10], labels[i:i+10], 'cuda')
  confidence, pred = get_predictions(probs)
  tot_pred[i:i+10] = pred

In [27]:
sum(labels.squeeze() == tot_pred.to('cuda')) / 16500.0

tensor(0.8973, device='cuda:0')

## CUDA Memory debugs

In [0]:
def pretty_size(size):
	"""Pretty prints a torch.Size object"""
	assert(isinstance(size, torch.Size))
	return " × ".join(map(str, size))

def dump_tensors(gpu_only=True):
	"""Prints a list of the Tensors being tracked by the garbage collector."""
	import gc
	total_size = 0
	for obj in gc.get_objects():
		try:
			if torch.is_tensor(obj):
				if not gpu_only or obj.is_cuda:
					print("%s:%s%s %s" % (type(obj).__name__, 
										  " GPU" if obj.is_cuda else "",
										  " pinned" if obj.is_pinned else "",
										  pretty_size(obj.size())))
					total_size += obj.numel()
			elif hasattr(obj, "data") and torch.is_tensor(obj.data):
				if not gpu_only or obj.is_cuda:
					print("%s → %s:%s%s%s%s %s" % (type(obj).__name__, 
												   type(obj.data).__name__, 
												   " GPU" if obj.is_cuda else "",
												   " pinned" if obj.data.is_pinned else "",
												   " grad" if obj.requires_grad else "", 
												   " volatile" if obj.volatile else "",
												   pretty_size(obj.data.size())))
					total_size += obj.data.numel()
		except Exception as e:
			pass        
	print("Total size:", total_size)

In [0]:
torch.cuda.empty_cache() 

In [46]:
dump_tensors()

/usr/local/lib/python3.6/dist-packages/torch/distributed/distributed_c10d.py:101: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: astor.all_symbols is deprecated.  Please use astor.symbol_data.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: astor.treewalk is deprecated.  Please use astor.tree_walk.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: astor.codegen is deprecated.  Please use astor.code_gen.


Parameter: GPU pinned 1024 × 1024
Parameter: GPU pinned 1024
Parameter: GPU pinned 1 × 1024
Parameter: GPU pinned 1024
Parameter: GPU pinned 1024
Parameter: GPU pinned 50265 × 1024
Parameter: GPU pinned 514 × 1024
Parameter: GPU pinned 1024 × 1024
Parameter: GPU pinned 1024
Parameter: GPU pinned 3 × 1024
Parameter: GPU pinned 3
Parameter: GPU pinned 4096 × 1024
Parameter: GPU pinned 4096
Parameter: GPU pinned 1024 × 4096
Parameter: GPU pinned 1024
Parameter: GPU pinned 1024
Parameter: GPU pinned 1024
Parameter: GPU pinned 4096 × 1024
Parameter: GPU pinned 4096
Parameter: GPU pinned 1024 × 4096
Parameter: GPU pinned 1024
Parameter: GPU pinned 1024
Parameter: GPU pinned 1024
Parameter: GPU pinned 1024 × 1024
Parameter: GPU pinned 1024
Parameter: GPU pinned 1024 × 1024
Parameter: GPU pinned 1024
Parameter: GPU pinned 1024 × 1024
Parameter: GPU pinned 1024
Parameter: GPU pinned 1024 × 1024
Parameter: GPU pinned 1024
Parameter: GPU pinned 1024
Parameter: GPU pinned 1024
Parameter: GPU pinne

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: FutureWarning: Index.data is deprecated and will be removed in a future version
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: FutureWarning: RangeIndex.data is deprecated and will be removed in a future version
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: FutureWarning: Series.data is deprecated and will be removed in a future version
